In [1]:
import requests
from bs4 import BeautifulSoup
import io
import time
import re
import pandas as pd

In [2]:
def sleeptime(hore,min,sec):
    return hore * 3600 + min * 60 + sec

In [3]:
url = "https://www.dcard.tw/f"
headers = {'User-Agent':'Mozilla/5.0(Windows NT 10.0;Win64;x64)AppleWebKit / 537.36 Chrome / 70.0.3538.102 Safari / 537.36'} 
resp = requests.get(url, headers = headers)
print(resp.status_code)

200


In [8]:
soup = BeautifulSoup(resp.text, 'html.parser')
dcard_title = soup.find_all('a', re.compile('sc-1v1d5rx-3 kPUUNB'))
dcard_title_s = []
for i, title in enumerate(dcard_title):
    print(str(i) + ". " + title.span.string)
    dcard_title_s.append(title.span.string)

0. #半熟大人交朋友  Lulu 黃路梓茵：我的人生就是一個「超前部署」
1. 在交友軟體上遇到親姐……？
2. 穿著比較露的衣服爭議
3. 時尚界震撼彈GUCCI宣布退出時裝週
4. 高清無碼😎（內有照片）
5. 終於見到卡友了~
6. 女宿外有人趴在欄杆上偷看
7. 年輕的雞（？？？？？
8. 胸部下擠出巨型粉刺
9. 星巴克美人魚試喝結果？！
10. 美咖真的無所不用其極欸
11. 只有我覺得Miyeon瀏海超正嗎
12. 當8＋9其實真的很爽
13. 這樣拒絕還不夠明顯嗎
14. 求救～～背心裡面內衣要怎麼穿
15. 女友是全世界最機掰的生物
16. 8+9最愛的食物
17. #台中#黑特#第一次做美甲做到滿手傷痕
18. 圭賢啊~這14年來辛苦啦☺️
19. <0528*更+抱歉,原意純屬分享> 一些讀書帳的日常
20. 閨蜜要我丟了我的比基尼
21. 壓麵包很奇怪？
22. 關於醫生的口頭禪
23. 喜歡警察制服的人是什麼感覺
24. 佈置風格分享｜乾淨木質調🕰
25. #更 SEULGI X IRENE 6/15 MONSTER 出道
26. 不小心看到男友電腦裡的大奶妹
27. Netflix真的很雷嗎？
28. 朋奉哥答應「浪子回頭」，合作夥伴：永遠忘不了
29. #分享 三個月過後我的臉幾乎恢復啦( ´▽` )ﾉ


In [9]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output
from transformers import BertForMaskedLM

PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.3.1


In [10]:
vocab = tokenizer.vocab
print(len(vocab))

21128


In [11]:
text = dcard_title_s[12]
print(text)
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
tokens[5] = '[MASK]'
ids[5] = 103
print(tokens[:20], '...')
print(ids[:20], '...')


當8＋9其實真的很爽
['當', '8', '##＋', '##9', '其', '[MASK]', '真', '的', '很', '爽'] ...
[4534, 129, 11267, 8160, 1071, 103, 4696, 4638, 2523, 4272] ...


In [12]:
tokens_tensor = torch.tensor([ids])
segments_tensors = torch.zeros_like(tokens_tensor)  # (1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()

In [13]:
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensors)
    predictions = outputs[0]
del maskedLM_model

In [14]:
# 將 [MASK] 位置的機率分佈取 top k 最有可能的 tokens 出來
masked_index = 5
k = 5
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

# 顯示 top k 可能的字。一般我們就是取 top 1 當作預測值
print("輸入 tokens ：", tokens[:10], '...')
print('-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    tokens[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), tokens[:10]), '...')

輸入 tokens ： ['當', '8', '##＋', '##9', '其', '[MASK]', '真', '的', '很', '爽'] ...
--------------------------------------------------
Top 1 (82%)：['當', '8', '##＋', '##9', '其', '實', '真', '的', '很', '爽'] ...
Top 2 ( 5%)：['當', '8', '##＋', '##9', '其', '後', '真', '的', '很', '爽'] ...
Top 3 ( 3%)：['當', '8', '##＋', '##9', '其', '中', '真', '的', '很', '爽'] ...
Top 4 ( 0%)：['當', '8', '##＋', '##9', '其', '上', '真', '的', '很', '爽'] ...
Top 5 ( 0%)：['當', '8', '##＋', '##9', '其', '后', '真', '的', '很', '爽'] ...
